## Two Site 1D Hubbard QITE
This notebook is a guide to implement QITE on the Fermi-Hubbard model with two sites

We will refer to Pauli matrices by their indices: $[I, X, Y, Z] \equiv [0, 1, 2, 3]$
For consistency in notation, we use the following mapping for 2-qubit operators, composed of a Pauli matrices acting on each qubit. This is the indexing used throughout the code.

In [1]:
pauli_indices = [0, 1, 2, 3]
pauli_pairs = [[i, j] for i in pauli_indices for j in pauli_indices]
print(pauli_pairs)

[[0, 0], [0, 1], [0, 2], [0, 3], [1, 0], [1, 1], [1, 2], [1, 3], [2, 0], [2, 1], [2, 2], [2, 3], [3, 0], [3, 1], [3, 2], [3, 3]]


We define a measurement function **measure** and a propagation function **propagate**

In **measure** we define circuits to measure the expectation value of any Pauli string

### Measure Pauli Expectation

In [2]:
from pyquil.gates import *
from pyquil.noise import estimate_bitstring_probs,correct_bitstring_probs
from pyquil import Program,get_qc

def measure(p, ro, idx_pair, qc, qbits):
    # Circuit to measure the expectation value of any Pauli string
    # For 2-qubit Pauli measurements, see https://docs.microsoft.com/en-us/quantum/concepts/pauli-measurements
    # initialize qc    
    qc = get_qc(qc)
    # measure II
    if idx_pair == [0, 0]:
        return 1
    # measure IX
    elif idx_pair == [0, 1]:
        p += SWAP(qbits[0], qbits[1])
        p += H(qbits[0])
    # measure IY
    elif idx_pair == [0, 2]:
        p += SWAP(qbits[0], qbits[1])
        p += RX(np.pi/2, qbits[0])
    # measure IZ
    elif idx_pair == [0, 3]:
        p += SWAP(qbits[0], qbits[1])

    # measure XI
    elif idx_pair == [1, 0]:
        p += H(qbits[0])
    # measure XX
    elif idx_pair == [1, 1]:
        p += H(qbits[0])
        p += H(qbits[1])
        p += CNOT(qbits[1], qbits[0])
    # measure XY
    elif idx_pair == [1,2]:
        p += H(qbits[0])
        p += RX(np.pi/2, qbits[1])
        p += CNOT(qbits[1], qbits[0])
    # measure XZ
    elif idx_pair == [1, 3]:
        p += H(qbits[0])
        p += CNOT(qbits[1], qbits[0])
    # measure YI
    elif idx_pair == [2, 0]:
        p += RX(np.pi/2, qbits[0])
    # measure YX
    elif idx_pair == [2, 1]:
        p += RX(np.pi/2, qbits[0])
        p += H(qbits[1])
        p += CNOT(qbits[1], qbits[0])
    # measure YY
    elif idx_pair == [2, 2]:
        p += RX(np.pi/2, qbits[0])
        p += RX(np.pi/2, qbits[1])
        p += CNOT(qbits[1], qbits[0])
    #measure YZ
    elif idx_pair == [2, 3]:
        p += RX(np.pi/2, qbits[0])
        p += CNOT(qbits[1], qbits[0])
    # measure ZI
    elif idx_pair == [3, 0]:
        return 1
    # measure ZX
    elif idx_pair == [3, 1]:
        p += H(qbits[1])
        p += CNOT(qbits[1], qbits[0])
    # measure ZY
    elif idx_pair == [3, 2]:
        p += RX(np.pi/2, qbits[1])
        p += CNOT(qbits[1], qbits[0])
    # measure ZZ
    elif idx_pair == [3, 3]:
        p += CNOT(qbits[1], qbits[0])
    # oops
    else:
        raise ValueError

    p += MEASURE(qbits[0], ro[0])
    exe = qc.compile(p)
    res = qc.run(exe)
    probs = estimate_bitstring_probs(res)
    # probs = correct_bitstring_probs(probs,[correction_matrix])
    return probs[0] - probs[1]

### Propagate the state

In **propagate**, we loop through the different values store in alist to construct the states. alist is stored as a list of lists and the indices are $a[timestep][gate]$. For a timestep $\Delta \tau$, the gate indices of $a$ correspond to:

$$
a[0]=e^{-ia[II]\Delta\tau\hat{I} \otimes \hat{I}} \text{,  } \; 
a[1]=e^{-ia[IX]\Delta\tau\hat{I} \otimes \hat{X}} \text{,  } \;
a[2]=e^{-ia[IY]\Delta\tau\hat{I} \otimes \hat{Y}} \text{,  } \;
a[3]=e^{-ia[ZZ]\Delta\tau\hat{I} \otimes \hat{Z}}
$$
$$
a[4]=e^{-ia[XI]\Delta\tau\hat{X} \otimes \hat{I}} \text{,  } \; 
a[5]=e^{-ia[XX]\Delta\tau\hat{X} \otimes \hat{X}} \text{,  } \;
a[6]=e^{-ia[XY]\Delta\tau\hat{X} \otimes \hat{Y}} \text{,  } \;
a[7]=e^{-ia[XZ]\Delta\tau\hat{X} \otimes \hat{Z}}
$$
$$
a[8]=e^{-ia[YI]\Delta\tau\hat{Y} \otimes \hat{I}} \text{,  } \; 
a[9]=e^{-ia[YX]\Delta\tau\hat{Y} \otimes \hat{X}} \text{,  } \;
a[10]=e^{-ia[YY]\Delta\tau\hat{Y} \otimes \hat{Y}} \text{,  } \;
a[11]=e^{-ia[YZ]\Delta\tau\hat{Y} \otimes \hat{Z}}
$$
$$
a[12]=e^{-ia[ZI]\Delta\tau\hat{Z} \otimes \hat{I}} \text{,  } \; 
a[13]=e^{-ia[ZX]\Delta\tau\hat{Z} \otimes \hat{X}} \text{,  } \;
a[14]=e^{-ia[ZY]\Delta\tau\hat{Z} \otimes \hat{Y}} \text{,  } \;
a[15]=e^{-ia[ZZ]\Delta\tau\hat{Z} \otimes \hat{Z}}
$$

The 0 index stores the coefficient for the identity matrix $\hat{I}$ on both qubits, which is a global phase for each qubit that we can ignore.

We can break down the matrix exponential of the Kronecker product of two Pauli matrices as:

$$e^{-i \theta \Delta\tau \bigotimes_{j} \sigma_{j}} = \cosh(-i \theta \Delta\tau) \bigotimes_{j} I + \sinh(-i \theta \Delta\tau) \bigotimes_{j} \sigma_{j} = \cos(\theta \Delta\tau) \bigotimes_{j} I - i \sin(\theta \Delta\tau) \bigotimes_{j} \sigma_{j}$$

For example,

$$e^{-ia[XY]\Delta\tau\hat{X} \otimes \hat{Y}} = \cos(a[XY] \Delta\tau) (I \otimes I) - i \sin(a[XY] \Delta\tau) (X \otimes Y)$$

To implement this in terms of gates on a quantum computer, refer to page 210 in Nielsen and Chuang: http://mmrc.amss.cas.cn/tlb/201702/W020170224608149940643.pdf

We need to apply a phase shift to the system. Phase is $e^{-i \Delta \tau}$ if the parity of the $n$ qubits in the computational basis is even, and $e^{i \Delta \tau}$ if odd. For 2 qubits, this is the matrix exponential of $i \Delta \tau Z \otimes Z$

$$ e^{-i \Delta \tau Z \otimes Z} =
\begin{pmatrix}
e^{-i \Delta \tau} & 0 & 0 & 0\\
0 & e^{i \Delta \tau} & 0 & 0\\
0 & 0 & e^{i \Delta \tau} & 0\\
0 & 0 & 0 & e^{-i \Delta \tau}\\
\end{pmatrix}
$$

The function **applyPhase** implements this

In [3]:
def applyPhase(p,qbits,angle):
    p += CNOT(qbits[0],qbits[1])
    # multiply angle by -1 to flip the phases the right way
    p += RZ(-angle,qbits[1]) # since angle was multiplied by -2 * t in update_alist
    p += CNOT(qbits[0],qbits[1])

In [4]:
def propagate(p,alist,qbits):
    # Circuit to propagate the state
    if len(alist) == 0:
        None
    else:
        for t in range(len(alist)):
            
            for i in range(1,16):
                angle = np.real(alist[t][i])
                idx_pair = pauli_pairs[i]
                
                # II... we skip this case because it's just a phase
                if idx_pair == [0, 0]:
                    continue
                # IX
                elif idx_pair == [0, 1]:
                    p += RX(angle,qbits[1])

                # IY
                elif idx_pair == [0, 2]:
                    p += RY(angle,qbits[1])

                # IZ
                elif idx_pair == [0, 3]:
                    p += RZ(angle,qbits[1])

                # XI
                elif idx_pair == [1, 0]:
                    p += RX(angle,qbits[0])
                    
                # XX
                elif idx_pair == [1, 1]:
                    p += H(qbits[0])
                    p += H(qbits[1])
                    applyPhase(p,qbits,angle)
                    p += H(qbits[1])
                    p += H(qbits[0])
                    
                # XY
                elif idx_pair == [1,2]:
                    p += H(qbits[0])
                    p += RX(np.pi/2, qbits[1])
                    applyPhase(p,qbits,angle)
                    p += RX(-np.pi/2, qbits[1])
                    p += H(qbits[0])
                    
                # XZ
                elif idx_pair == [1, 3]:
                    p += H(qbits[0])
                    applyPhase(p,qbits,angle)
                    p += H(qbits[0])
                    
                # YI
                elif idx_pair == [2, 0]:
                    p += RY(angle,qbits[0])
                    
                # YX
                elif idx_pair == [2, 1]:
                    p += RX(np.pi/2, qbits[0])
                    p += H(qbits[1])
                    applyPhase(p,qbits,angle)
                    p += H(qbits[1])
                    p += RX(-np.pi/2, qbits[0])
                    
                # YY
                elif idx_pair == [2, 2]:
                    p += RX(np.pi/2, qbits[0])
                    p += RX(np.pi/2, qbits[1])
                    applyPhase(p,qbits,angle)
                    p += RX(-np.pi/2, qbits[1])
                    p += RX(-np.pi/2, qbits[0])
                    
                # YZ
                elif idx_pair == [2, 3]:
                    p += RX(np.pi/2, qbits[0])
                    applyPhase(p,qbits,angle)
                    p += RX(-np.pi/2, qbits[0])
                    
                # ZI
                elif idx_pair == [3, 0]:
                    p += RZ(angle,qbits[0])
                    
                # ZX
                elif idx_pair == [3, 1]:
                    p += H(qbits[1])
                    applyPhase(p,qbits,angle)
                    p += H(qbits[1])
                    
                # ZY
                elif idx_pair == [3, 2]:
                    p += RX(np.pi/2, qbits[1])
                    applyPhase(p,qbits,angle)
                    p += RX(-np.pi/2, qbits[1])
                    
                # ZZ
                elif idx_pair == [3, 3]:
                    applyPhase(p,qbits,angle)
                    
                # oops
                else:
                    raise ValueError
                

We now want to obtain the coefficients a[m] at the current time step and append to alist. For the 1-qubit case, we need to construct the matrix $S_{ij} = \langle \psi | \sigma_{i} \sigma_{j} | \psi \rangle$ and the vector $b_{i}$. Well, our earlier functions allow us to measure the expectation values of the different pauli matrices. How do we obtain $S_{ij}$ from a list of $\langle \psi | \sigma | \psi \rangle$? We can exploit the fact that up to some coefficients, $\sigma_{i}\sigma_{j} = c_{ij}\sigma_{ij}$. For example, $\sigma_{x}\sigma_{y} = i\sigma_{z}$. We will need a matrix to keep track of what pauli matrix and coefficient we get for $\sigma_{i}\sigma_{j}$. These are stored in the matrices **idx** and **coeff** below.


In [5]:
import numpy as np

# To keep track of Lie algebra. Let P represent some Pauli operator. We want to know PiPj = cijPij.

idx = np.zeros([4,4],dtype=int)
idx[0,0] = 0
idx[0,1] = 1
idx[0,2] = 2
idx[0,3] = 3
idx[1,0] = 1
idx[1,1] = 0
idx[1,2] = 3
idx[1,3] = 2
idx[2,0] = 2
idx[2,1] = 3
idx[2,2] = 0
idx[2,3] = 1
idx[3,0] = 3
idx[3,1] = 2
idx[3,2] = 1
idx[3,3] = 0

coeff = np.zeros([4,4],dtype=complex)
coeff[0,0] = 1
coeff[0,1] = 1
coeff[0,2] = 1
coeff[0,3] = 1
coeff[1,0] = 1
coeff[1,1] = 1
coeff[1,2] = 1j
coeff[1,3] = -1j
coeff[2,0] = 1
coeff[2,1] = -1j
coeff[2,2] = 1
coeff[2,3] = 1j
coeff[3,0] = 1
coeff[3,1] = 1j
coeff[3,2] = -1j
coeff[3,3] = 1

print(idx)
print(coeff)

[[0 1 2 3]
 [1 0 3 2]
 [2 3 0 1]
 [3 2 1 0]]
[[ 1.+0.j  1.+0.j  1.+0.j  1.+0.j]
 [ 1.+0.j  1.+0.j  0.+1.j -0.-1.j]
 [ 1.+0.j -0.-1.j  1.+0.j  0.+1.j]
 [ 1.+0.j  0.+1.j -0.-1.j  1.+0.j]]


For the 2-qubit case, we can reuse these matrices. For example,
$(X \otimes X) (X \otimes Y) = XX \otimes XY = I \otimes i Z$

We need to calculate:

$$S_{ij} = \langle \psi | Q_{i} Q_{j} | \psi \rangle = \langle \psi | ( \sigma_{i0} \otimes \sigma_{i1} ) ( \sigma_{j0} \otimes \sigma_{j1} ) | \psi \rangle = \langle \psi | \left( \sigma_{i0} \sigma_{j0} \otimes \sigma_{i1} \sigma_{j1} \right) | \psi \rangle$$

And the vector 
$$b_{i} = i \langle \psi | Q_{i} | \Delta_{0} \rangle - h.c. = i \langle \psi | {( \sigma_{i0} \otimes \sigma_{i1})} | \Delta_{0} \rangle - h.c.$$

where $| \Delta_{0} \rangle = \frac{(| \psi' \rangle - | \psi \rangle)}{\Delta \tau}$

Using this, we can construct the S matrix and b vector. We use the **idx** and **coeff** matrices and the function np.linalg.std to solve for x in **update_alist**.

In [6]:
# a matrix should be 16 x 16
def update_alist(sigma_expectation,alist,db,delta,hm):
    '''
    Obtain A[m]
    To do this, we compute the S matrix and the b vector. We also need to compute the norm c
    Details:
    Each local Hamiltonian term hm can be a sum of many 2-qubit operators...
     ... see the construction of the hm_list to see why... for each term in hm (hm[i]):
    hm[i][1][0] is the multiplicative constant in the Hamiltonian
    hm[i][0][0] is the index of the Pauli matrix for qubit 0
    hm[i][0][1] is the index of the Pauli matrix for qubit 1
    '''
    # c is the squared norm
    c = 1
    for i in range(len(hm)):
        c -= 2 * db * hm[i][1][0] * sigma_expectation[hm[i][0][0], hm[i][0][1]]
    # c is now the norm, after we take its square root
    c = np.sqrt(c)
        
    # Initialize S matrix
    S = np.zeros([16, 16], dtype=complex)
    # Initialize b vector
    b = np.zeros([16], dtype=complex)
    
    # iterate through all Pauli pair combinations
    for i in range(16):
        idx_i = pauli_pairs[i]
        # Step 1: Obtain S matrix
        # iterate through all Pauli pair combinations
        for j in range(16):
            idx_j = pauli_pairs[j]
            idx_q0, idx_q1 = [idx[idx_i[0], idx_j[0]], idx[idx_i[1], idx_j[1]]]
            coeff_q0, coeff_q1 = [coeff[idx_i[0], idx_j[0]], coeff[idx_i[1], idx_j[1]]]
            coeff_val = coeff_q0 * coeff_q1
            S[i, j] = sigma_expectation[idx_q0, idx_q1] * coeff_val
            
        # Step 2: Obtain b vector
        b[i] += (sigma_expectation[idx_i[0], idx_i[1]] / c - sigma_expectation[idx_i[0], idx_i[1]]) / (db)
        # iterate through hm terms
        for j in range(len(hm)):
            idx_q0, idx_q1 = [idx[idx_i[0], hm[j][0][0]], idx[idx_i[1], hm[j][0][1]]]
            coeff_q0, coeff_q1 = [coeff[idx_i[0], hm[j][0][0]], coeff[idx_i[1], hm[j][0][1]]]
            coeff_val = coeff_q0 * coeff_q1
            b[i] -= hm[j][1][0] * coeff_val * sigma_expectation[idx_q0, idx_q1] / c
        b[i] = 1j * b[i] - 1j * np.conj(b[i])
        
    # Step 3: Add regularizer... 16 x 16 matrix with 1s on main diagonal and 0s elsewhere
    dalpha = np.eye(16)*delta

    # Step 4: Solve for linear equation, the solution is multiplied by -2 because of the definition of unitary rotation gates is exp(-i theta/2)
    x = np.linalg.lstsq(S + np.transpose(S) + dalpha, -b, rcond=-1)[0]
    alist.append([])
    for i in range(len(x)):
        alist[-1].append(x[i] * -2 * db)
    return c



Now we implement the full QITE protocol. It will be good to have a big picture of what should be done. We step through imaginary time and at each time step, we should first measure the expectation values of the pauli matrices $\sigma$. This is indicated in the first two lines of the for loop. Using the relevant expectation values, we obtain the coefficients $a[m]$ in equation (2.5) of Lecture 12 using the update rule in equation (2.9-2.11). We store it in a list and use this to propagate our state. Note we have to always reconstruct our state for each new measurements we make. Finally, we measure the current energy values. Let us construct the required functions starting with **get_expectation**. We make use of the **measure** function we constructed earlier. The key here is to propagate our state using the coefficients in alist. We do this using the **propagate** function.

In [7]:
import matplotlib.pyplot as plt
# from pyquil.gates import *
# from pyquil.noise import estimate_bitstring_probs
# from pyquil import Program,get_qc

def ansatz(p, qbits):
    None

def measure_energy(alist, shots, qc, qbits, hm_list):
    # Measure the energy at the end of each time step
    Energy = 0
    Nterms = len(hm_list)
    for i in range(len(hm_list)):
        hm = hm_list[i]
        # For each Pauli matrix pair (2-qubit operator) in the Hamiltonian, 
        for j in range(len(hm)):
            # Initialize state and propagate
            p = Program()
            p.wrap_in_numshots_loop(shots)
            ro = p.declare('ro', 'BIT', 1)
            ansatz(p, qbits)
            propagate(p, alist, qbits)
            # pauli pair to measure
            pauli_pair = [hm[j][0][0], hm[j][0][1]]
            # energy contribution of this term
            tmp = hm[j][1][0] * measure(p, ro, pauli_pair, qc, qbits)
            print("pauli_pair {} energy contribution: {}".format(pauli_pair, tmp))
            Energy += tmp

    return Energy

# use this to avoid the extra time complexity of propagating and measuring again... the values we want are already in sigma_expectation
def get_energy_from_sigma(sigma_expectation, hm_list):
    Energy = 0
    Nterms = len(hm_list)
    for i in range(len(hm_list)):
        hm = hm_list[i]
        # For each Pauli matrix pair (2-qubit operator) in the Hamiltonian, 
        for j in range(len(hm)):
            # pauli pair to retrieve the measurement for in sigma_expectations
            pauli_pair = [hm[j][0][0], hm[j][0][1]]
            # energy contribution of this term
            tmp = hm[j][1][0] * sigma_expectation[pauli_pair[0],pauli_pair[1]]
            print("pauli_pair {} energy contribution: {}".format(pauli_pair, tmp))
            Energy += tmp

    return Energy

def get_expectation(alist, shots, qc, qbits):
    # Obtain the expectation values of the Pauli string at each time step
    sigma_expectation = np.zeros([4, 4], dtype=complex)
    for i in range(16):
        p = Program()
        p.wrap_in_numshots_loop(shots)
        ro = p.declare('ro','BIT',1)
        ansatz(p,qbits)
        propagate(p, alist, qbits)
        idx_q0, idx_q1 = pauli_pairs[i]
        sigma_expectation[idx_q0, idx_q1] = measure(p, ro, pauli_pairs[i], qc, qbits)
    return sigma_expectation

def qite_step(alist, shots, qc, qbits, db, delta, hm_list):
    # Every qite step we start from scratch and re-propagate all time steps in alist.
    # Then, we get the expectations of the Pauli strings in the current state (after propagation)
    # These two steps are done in get_expectation
    sigma_expectation = get_expectation(alist, shots, qc, qbits)
    energy = get_energy_from_sigma(sigma_expectation, hm_list)
    for j in range(len(hm_list)):
        update_alist(sigma_expectation, alist, db, delta, hm_list[j]) # = norm
    return alist, sigma_expectation, energy

def qite(qc, qbits, shots, db, delta, N, hm_list):
    E = np.zeros([N],dtype=complex)
    alist = []
    QITE_expectations = [None] * N
#     E[0] = measure_energy(alist, shots, qc, qbits, hm_list)
#     print("Initial energy: ", E[0])
    # Qite main loop
    for i in range(0,N):
        print("QITE step: ", i)
        # sigma_expectation is Pauli expectations at the start of each step
        alist, sigma_expectation, energy = qite_step(alist, shots, qc, qbits, db, delta, hm_list)
        QITE_expectations[i] = sigma_expectation.real.flatten().tolist()
        print("Pauli expectations: ", sigma_expectation.real.flatten().tolist())
#         E[i] = measure_energy(alist, shots, qc, qbits, hm_list)
        E[i] = energy
#         print("a list: ", alist)
        print("Energy: ", E[i])
    return E, QITE_expectations

### Running QITE with 2-site 1D Hubbard Hamiltonian

The 2-site 1D Hubbard Hamiltonian, for a half-filled lattice (2 fermions) is:

$$ H = -t \sum_{\sigma} \left( a_{1 \sigma}^{\dagger} a_{2 \sigma} + a_{2 \sigma}^{\dagger} a_{1 \sigma} \right) + U \sum_{i=1}^{2} n_{i \uparrow} n_{i \downarrow} $$

We can solve for the ground state energy of this Hamiltonian exactly, since the Hilbert space is small enough. We expect the ground state wavefunction to be symmetric with spin up and spin down components, and the most general guess is:

$$| \psi \rangle = \alpha \left( a_{1 \uparrow}^{\dagger} a_{1 \downarrow}^{\dagger} + a_{2 \uparrow}^{\dagger} a_{2 \downarrow}^{\dagger} \right) |0 \rangle + \beta \left( a_{1 \uparrow}^{\dagger} a_{2 \downarrow}^{\dagger} + a_{1 \uparrow}^{\dagger} a_{2 \downarrow}^{\dagger} \right) |0 \rangle$$

Using the time independent Schrodinger equation $H | \psi \rangle = E | \psi \rangle$ we get the coupled equations:

$$ (E-U) \alpha + 2 t \beta = 0$$
$$2t \alpha + E \beta = 0$$

There are two solutions; the one with the lower energy is the ground state energy:

$$E_{0} = \frac{1}{2} \left( U - \sqrt{U^{2} + 16 t^{2}} \right)$$

For our simulation, we run QITE with $t=1$ and $U=2$, so our goal is to match the exact ground state energy of $E_{0} = -1.23607$

We run QITE with a two qubit QVM using the 2-site 1D Hubbard Hamiltonian, with fermionic operators mapped to Pauli matrices by the Jordan-Wigner transformation.

$$H = -t \left( X \otimes I + I \otimes X \right) + \frac{U}{2} \left( I + Z \otimes Z \right)$$

In [ ]:
# ---- input parameters for qite
N = 40
shots = 1000 #8192
db = 0.05
# use a 2-qubit qvm
qc = '2q-qvm'
qbits = [0, 1]
# qbits = [1, 0]
# kinetic energy contribution
t = 1.
# potential energy contribution
U = 2.
print("t = ", t, ", U = ", U)
# construct hm_list to represent H = -t(X \otimes I + I \otimes X) + U/2 (I + Z \otimes Z)
hm_list = []
# the first term (hopping kinetic term) of H
hm_list.append([])
hm_list[0].append([[1, 0], [-t]])
hm_list[0].append([[0, 1], [-t]])

hm_list[0].append([[0, 0], [U/2]])
hm_list[0].append([[3, 3], [U/2]])
# # the second term (onsite potential term) of H
# hm_list.append([])
# hm_list[1].append([[0, 0], [U/2]])
# hm_list[1].append([[3, 3], [U/2]])
print("hm list: ", hm_list)
delta = 0.01
print("running QITE with N = {} steps and imaginary time steps db = {}".format(N, db))
E, QITE_expectations = qite(qc, qbits, shots, db, delta, N, hm_list)

t =  1.0 , U =  2.0
hm list:  [[[[1, 0], [-1.0]], [[0, 1], [-1.0]], [[0, 0], [1.0]], [[3, 3], [1.0]]]]
running QITE with N = 40 steps and imaginary time steps db = 0.05
QITE step:  0
pauli_pair [1, 0] energy contribution: (0.014000000000000012-0j)
pauli_pair [0, 1] energy contribution: (0.016000000000000014-0j)
pauli_pair [0, 0] energy contribution: (1+0j)
pauli_pair [3, 3] energy contribution: (1+0j)
Pauli expectations:  [1.0, -0.016000000000000014, -0.02200000000000002, 1.0, -0.014000000000000012, -0.05600000000000005, 0.03200000000000003, 0.020000000000000018, -0.010000000000000009, 0.0040000000000000036, -0.09800000000000003, -0.04200000000000004, 1.0, -0.0020000000000000018, -0.01200000000000001, 1.0]
Energy:  (2.0300000000000002+0j)
QITE step:  1
pauli_pair [1, 0] energy contribution: (-0.08600000000000002+0j)
pauli_pair [0, 1] energy contribution: (0.10200000000000004-0j)
pauli_pair [0, 0] energy contribution: (1+0j)
pauli_pair [3, 3] energy contribution: (0.998+0j)
Pauli expect

In [ ]:
# some trouble with numpy sometimes because of imaginary part... so plot only real part of E
plt.figure(figsize=[8,4], dpi=150)
plt.plot(np.arange(0,N+1)*db,E.real,'g-o',label='QITE')
plt.axhline(y=-1.23607, color='r',linestyle='--',label="Exact")
plt.title("QITE on 2-site 1D Hubbard Model")
plt.xlabel("Imaginary time")
plt.ylabel("Energy")
plt.grid()
plt.legend(bbox_to_anchor=(1.0,1.0))
plt.show()

In [ ]:
def get_diff_between_lists(a,b):
    assert len(a) == len(b), print("go away")
    diff = [0]*len(a)
    for i in range(len(a)):
        diff[i] = a[i] - b[i]
        if diff[i] < 0.001:
            diff[i] = 0
    return diff

In [ ]:
classical_QITE_expectations = [[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0], [0.9999999999999998, 0.11086108265002184, -2.3701636405366485e-17, 0.9937924042757851, 0.11086108265002173, 0.003106211876230596, 2.0536618701879933e-16, 0.11120705306559495, -6.099097543193006e-16, 1.3976850585736573e-16, 0.009299331019978764, -6.290893652905017e-16, 0.993792404275785, 0.11120705306559506, -4.105183473401435e-17, 0.9875944571037903], [1.0, 0.22855138352124413, 2.928964092544699e-16, 0.9733007472581157, 0.22891344515533618, 0.03224095094282042, 1.8979905581836617e-16, 0.22762207039851495, -3.803684151432205e-16, 3.249036198626321e-17, 0.021211328985208344, -4.0481511550415014e-16, 0.9732156565478156, 0.2272579513880564, 2.6460422348989474e-16, 0.9465476523440426], [0.9999999999999998, 0.3486496850725175, 4.988857016394855e-16, 0.9364943964684751, 0.34976673338301983, 0.08901950673898747, 9.88732432129611e-16, 0.3403438521309831, -7.344110575699351e-16, 5.416650098933578e-16, 0.03770467453752445, -1.0059565964818992e-15, 0.9360777690644033, 0.33919577356313513, 1.930936985574974e-16, 0.8732751338584406], [0.9999999999999996, 0.4652937955485409, 6.985365568213624e-16, 0.8831110226607517, 0.467459547353887, 0.17128538081358738, 1.0971424777018496e-15, 0.43908581417640685, -8.226348750649761e-16, 3.4340436752430833e-16, 0.06013821978669487, -1.1600212377996855e-15, 0.881966533440235, 0.4367793950893569, 2.666071997265018e-16, 0.7685735694213492], [1.0, 0.5723063246438079, 1.4095900790672458e-15, 0.815222581445681, 0.5755959493066705, 0.272747511827399, 1.6583831715502061e-15, 0.5145844003892328, -1.204418695693232e-15, 1.3849526610936902e-17, 0.08875592078129847, -1.6406002031813274e-15, 0.8129032474212515, 0.5109020820989113, 6.912639791385468e-16, 0.638489127275281], [0.9999999999999999, 0.6646580900382794, 1.7505908162398475e-15, 0.7370302700719432, 0.6688927194611752, 0.38413473487351335, 1.4008276541844669e-15, 0.5611363292072319, -1.9927030996724504e-15, -1.4429745823305e-15, 0.12253980718258425, -1.6934656303365623e-15, 0.7331892835465399, 0.556081726422775, 1.4427009156835793e-15, 0.4933108993226274], [1.0, 0.7395830987487371, 1.836396373567694e-15, 0.6538877897754687, 0.744401552643122, 0.49552225598905, 1.7233337902683843e-15, 0.5779610400578756, -2.870889767355274e-15, -2.430464080221474e-15, 0.15952303416048785, -2.0895131462992655e-15, 0.6483970465654059, 0.5717415956549556, 1.560027264399836e-15, 0.34493169790617173], [0.9999999999999997, 0.7968474186455121, 1.4650595389024476e-15, 0.5710276035604153, 0.801836106432342, 0.5986967311313995, 1.788612432404155e-15, 0.568739864156881, -3.021861642493027e-15, -2.675924310455611e-15, 0.1973871003131476, -2.0642471748059024e-15, 0.5640010558956328, 0.5616846981221986, 1.1123419199100494e-15, 0.20388515962864004], [1.0000000000000002, 0.8382099665893585, 1.1797968631146324e-15, 0.49257572126794286, 0.843016302702924, 0.6885053551905532, 1.7031789733875143e-15, 0.5398241132299215, -3.0275653279538558e-15, -2.9103048400787586e-15, 0.2340367721697446, -1.7545214218461642e-15, 0.48430393623163903, 0.5322870792075425, 9.34435599546793e-16, 0.07742078975182273], [1.0000000000000004, 0.8665112350848125, 9.175934342031215e-16, 0.4211479656910236, 0.870898899910587, 0.7629249870789886, 1.5977658696259378e-15, 0.49819978765762396, -2.9322846887940178e-15, -2.9672553640229296e-15, 0.26794154300165174, -1.4412653712219545e-15, 0.4119981015349758, 0.49048940364174864, 7.567553804379106e-16, -0.030907135902440597], [1.0000000000000004, 0.8848374848047438, 8.616184643597402e-16, 0.3579530289820156, 0.8886875434753677, 0.8222941839329236, 1.6838045928122369e-15, 0.4500334204538444, -2.833460315846485e-15, -2.981774317952145e-15, 0.2982151145180539, -1.2627977430439621e-15, 0.3482846472997359, 0.4423820793304001, 6.035953387072321e-16, -0.12055100978021299], [1.0000000000000004, 0.8959816998271508, 8.781250094042582e-16, 0.30315421334754006, 0.8992650606858686, 0.8683194467605058, 1.7659680480892717e-15, 0.4000166299697713, -2.590456203920735e-15, -2.839912082891974e-15, 0.32452167339714705, -1.0915815498031962e-15, 0.29327126372209955, 0.39257975152466074, 4.456994046037616e-16, -0.19288206065489202], [1.0000000000000009, 0.9022059002864324, 6.12970205935703e-16, 0.25626944423773956, 0.9049499152308078, 0.9032516066360607, 1.5453338658674278e-15, 0.3513137765930205, -2.5827303197750887e-15, -2.8743675722576476e-15, 0.3469156748237669, -7.886539487080091e-16, 0.24640447537600804, 0.34418382742287096, 4.484905828906733e-16, -0.2502062026894991], [1.0000000000000007, 0.9052093035172524, 6.425650630449969e-16, 0.21649360297066123, 0.9074702995617425, 0.9293573722241923, 1.5564783654972037e-15, 0.305816703235479, -2.310374173250692e-15, -2.661532347039173e-15, 0.36568652791509026, -6.286916384390221e-16, 0.2068115536296955, 0.2990407908050792, 3.625517178835941e-16, -0.2950800922597039], [1.0000000000000004, 0.9062004977700162, 7.196724174199694e-16, 0.1829143905450175, 0.908045256705819, 0.9486501204611674, 1.5882313390632266e-15, 0.2644846323472685, -2.0436886455127104e-15, -2.462283462020053e-15, 0.38123874878207287, -4.741735096456115e-16, 0.17352471927982938, 0.25807997729494275, 3.2629233334856574e-16, -0.32992201668501275], [1.0000000000000007, 0.9060033191604836, 6.801167824139079e-16, 0.1546371531210644, 0.9074982836531782, 0.9627940454481952, 1.5322823689235913e-15, 0.2276534590726958, -1.8940627406457802e-15, -2.318860979664562e-15, 0.39401184822895974, -3.953840691863225e-16, 0.1456074469995832, 0.22161899044105515, 2.462533640005291e-16, -0.3568359402336385], [1.0000000000000004, 0.9051593072135236, 7.057791303054859e-16, 0.13084636257644844, 0.9063651652032635, 0.9731040629859491, 1.5172290653397574e-15, 0.19527455865889098, -1.723210339107627e-15, -2.176631537562836e-15, 0.40443276074676193, -2.938732563740517e-16, 0.12221427633797219, 0.18959881856370164, 2.253277887448331e-16, -0.37756386917356155], [1.0000000000000002, 0.9040121452758717, 7.267197772817353e-16, 0.11082852121799064, 0.9049817009345766, 0.980588835295876, 1.4940941208084771e-15, 0.16708230067205482, -1.559835410275487e-15, -2.0283845226150428e-15, 0.41289112375824977, -2.3646721758148014e-16, 0.10261111487168947, 0.16174849296890337, 1.8159956575841345e-16, -0.39350418802834863], [1.0000000000000004, 0.9027715049776259, 7.057456155797775e-16, 0.09397407701550847, 0.9035493595756945, 0.986006845879045, 1.4307045681151677e-15, 0.14270398634523462, -1.4252388130770802e-15, -1.8858099593362802e-15, 0.41972905861938625, -2.0223939878860766e-16, 0.08617466078167654, 0.1376930380547296, 1.3052055599554532e-16, -0.4057575410039991], [1.0000000000000002, 0.9015588151915157, 6.921583527389474e-16, 0.07976931374641993, 0.902181937049083, 0.9899207356589572, 1.3771774479193962e-15, 0.12172815608954457, -1.3130378103753206e-15, -1.7690931610986238e-15, 0.42523941413619687, -1.558154121712636e-16, 0.0723822708090111, 0.11702048409084082, 1.1120353433824773e-16, -0.41517942960304544], [0.9999999999999998, 0.9004389138251871, 6.816902046341742e-16, 0.06778443349984348, 0.9009375701858489, 0.9927439584701208, 1.3247118502738638e-15, 0.10374474125409366, -1.2039705957283462e-15, -1.653111479923611e-15, 0.4296685153050222, -1.2311836222904395e-16, 0.06079853273884181, 0.09932121372687791, 9.07223766934263e-17, -0.42242962861456335], [0.9999999999999998, 0.8994413659301261, 6.670625001215921e-16, 0.057661056710332836, 0.8998401369395324, 0.9947783526373009, 1.2707871214241756e-15, 0.08836703550742647, -1.1081598306430456e-15, -1.5469315425409162e-15, 0.43322099648192997, -1.0008647720492092e-16, 0.05106168974727279, 0.08420945115166872, 7.119523205533797e-17, -0.42801459821994137], [1.0, 0.8985745168262607, 6.458378705787689e-16, 0.04910063320435748, 0.8988932559262157, 0.9962432628136554, 1.2104713597302098e-15, 0.07524235760141063, -1.019702398730365e-15, -1.4438222025847494e-15, 0.43606532254856434, -8.040751717647524e-17, 0.042871306499609935, 0.0713336464603711, 5.621905251173916e-17, -0.4323221314402419], [0.9999999999999999, 0.8978345748522203, 6.248372827593344e-16, 0.041854324119528274, 0.8980892591405094, 0.9972975719005281, 1.1528437662967752e-15, 0.06405593038345653, -9.3984130178042e-16, -1.3486387076502752e-15, 0.4383392427702888, -6.875137773437411e-17, 0.03597764394760178, 0.06038020067711031, 4.030289837467586e-17, -0.4356488425126853], [1.0, 0.8972113606329961, 6.030852983372866e-16, 0.03571444775859833, 0.8974148158851971, 0.9980561053172884, 1.0973929774313454e-15, 0.05453086265166224, -8.696023348126793e-16, -1.2629766897265485e-15, 0.44015480523605144, -5.309315206430144e-17, 0.030172763502064215, 0.051073367425540916, 3.4078013909716986e-17, -0.438221587064756], [0.9999999999999999, 0.8966918563274616, 5.80293239966561e-16, 0.030507372696974566, 0.8968543612148598, 0.9986017144271706, 1.0429213717596441e-15, 0.046426029125573975, -8.055027599632277e-16, -1.1826025797334546e-15, 0.4416027796641895, -4.3653881545014376e-17, 0.025283191308593078, 0.0431730952590087, 2.5982365859590172e-17, -0.4402139691282435], [1.0000000000000002, 0.896262324065119, 5.564514179272063e-16, 0.026087665382973413, 0.8963921065053782, 0.9989941118737063, 9.887800627219642e-16, 0.039532938457341016, -7.46082484465558e-16, -1.1063279386051783e-15, 0.4427564569560313, -3.4303311146525145e-17, 0.021163913120996886, 0.036471881724570054, 2.124427792243215e-17, -0.4417589764094444], [1.0000000000000004, 0.8959095079674139, 5.321284063443854e-16, 0.022333288246213945, 0.8960131490804393, 0.9992762990387419, 9.372943366196349e-16, 0.033672232984745365, -6.942358134901027e-16, -1.0377551689932904e-15, 0.4436748559133127, -2.655608311552691e-17, 0.01769347069561905, 0.03079127174293428, 1.782186774277783e-17, -0.44295861461248076], [1.0000000000000007, 0.8956212597007507, 5.08045494341813e-16, 0.019141661342629845, 0.8957040205513205, 0.9994792252004283, 8.875304800246917e-16, 0.028690186434353337, -6.460820771846474e-16, -9.730066289918074e-16, 0.4444053959765631, -2.1670716531940302e-17, 0.014769955826437448, 0.025978360105998577, 1.3760120128003902e-17, -0.4438912393530693], [1.0000000000000009, 0.8953868098143605, 4.853686165180011e-16, 0.01642642728938648, 0.8954528946147288, 0.9996251564354622, 8.412772354931055e-16, 0.024455395973980498, -6.024407473269941e-16, -9.137066655238676e-16, 0.44498610461111043, -1.8393588390715405e-17, 0.01230773046238609, 0.021902490905954752, 9.932150743977927e-18, -0.44461713239395023], [1.0000000000000004, 0.8951968285519338, 4.610634274629878e-16, 0.014114786929179002, 0.8952495960925901, 0.9997301072770135, 7.948764319146627e-16, 0.02085576295275282, -5.626242341421734e-16, -8.576676298594647e-16, 0.44544742780770924, -1.5757216955324324e-17, 0.010234732803088453, 0.018452246308672315, 6.853209568824571e-18, -0.44518274371567795], [1.0000000000000004, 0.8950433678266064, 4.3908896937347236e-16, 0.01214529813564208, 0.8950855009983107, 0.9998055936822916, 7.515827082888824e-16, 0.017795796745319503, -5.241857433393594e-16, -8.041915035857945e-16, 0.44581370710511276, -1.2461841286076604e-17, 0.008490256757049452, 0.015532758119462087, 5.5797880058943516e-18, -0.44562392140436774], [0.9999999999999999, 0.8949197418977373, 4.173628718713552e-16, 0.010466051264436449, 0.8949533834364365, 0.9998598956169247, 7.102650139232886e-16, 0.01519424212835141, -4.8940325541100445e-16, -7.548053420609265e-16, 0.44610437941429953, -9.598506161431254e-18, 0.007023114911287348, 0.013063342139066236, 4.80272116725722e-18, -0.4459683739547371], [0.9999999999999998, 0.894820382039279, 3.958833912913423e-16, 0.009033151956570673, 0.8948472431276501, 0.999898965866025, 6.70508964463001e-16, 0.012982012402448784, -4.57380892657158e-16, -7.085369049268281e-16, 0.4463349482763007, -7.10955349604567e-18, 0.005790113473508107, 0.010975437384584563, 4.381609456741659e-18, -0.4462375502364869], [0.9999999999999998, 0.8947406861315452, 3.755014785752073e-16, 0.0078094557642161, 0.8947621332316169, 0.9999270839059321, 6.333381937066305e-16, 0.011100401708523333, -4.2820038061914563e-16, -6.659124927047896e-16, 0.4465177678228692, -6.333606588769722e-18, 0.004754782149357095, 0.00921082363554876, 2.844179132217998e-18, -0.44644807723044394], [0.9999999999999999, 0.8946768749531605, 3.5600232839521177e-16, 0.006763509976744575, 0.8946939992041787, 0.9999473262027516, 5.981030407502136e-16, 0.0094995467668742, -4.010296134895363e-16, -6.259321980378477e-16, 0.4466626740134451, -5.2491343417657284e-18, 0.0038863133161980146, 0.007720087613632742, 2.0807509276650632e-18, -0.4466128614754289], [1.0000000000000002, 0.8946258612062301, 3.3767218321509647e-16, 0.005868666638312314, 0.8946395338721317, 0.9999619043994166, 5.648503123387468e-16, 0.008137108192524889, -3.752873483403374e-16, -5.880962045861956e-16, 0.44677749183782733, -4.363158127577711e-18, 0.003158673796946704, 0.00646130805493228, 1.489006810962078e-18, -0.44674193437741505], [1.0000000000000002, 0.8945851337856079, 3.198520039364534e-16, 0.005102337553846814, 0.8945960505473594, 0.9999724084641499, 5.331827937865207e-16, 0.0069771431556059915, -3.516335014801999e-16, -5.528225373716343e-16, 0.4468684421183408, -3.614937901480056e-18, 0.0025498595635450594, 0.005398931551170771, 1.0578615142523321e-18, -0.4468431020874917], [1.0000000000000004, 0.8945526577152771, 3.026540302506611e-16, 0.004445367474452455, 0.8945613740472637, 0.9999799814891124, 5.029629799655129e-16, 0.0059891435877703625, -3.295259901570334e-16, -5.195675388129995e-16, 0.44694046726891057, -3.1046140523217948e-18, 0.002041269242496979, 0.0045028134596468705, 6.250110539923307e-19, -0.4469224459944031]]

In [ ]:
assert len(QITE_expectations) == len(classical_QITE_expectations), print("...")

diff_matrix = [0] * N
# should be N
for i in range(N):
    a = QITE_expectations[i]
    b = classical_QITE_expectations[i]
    diff_matrix[i] = get_diff_between_lists(a,b)

print(np.array(diff_matrix))

In [ ]:
a = [1.0, -0.010000000000000009, 0.04400000000000004, 1.0, -0.0020000000000000018, -0.028000000000000025, 0.03600000000000003, 0.05600000000000005, -0.014000000000000012, -0.020000000000000018, 0.02400000000000002, 0.040000000000000036, 1.0, -0.040000000000000036, 0.008000000000000007, 1.0]
b = classical_QITE_expectations[0]
get_diff_between_lists(a,b)